In [ ]:
from os import walk
import matplotlib.transforms
import csv
import ast
import pyedflib
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy
from scipy.signal import oaconvolve
import time
import copy
import ast


In [ ]:
startfilename = "acq_000058160.edf.csv"
numberOfFiles = 1
fileoffset = 1234
savemetrics = False

AAnnopath = "C:\\Users\\aaron\\OneDrive\\Dokumente\\Uni\\Studienarbeit\\automatische Annotationen\\"
MAnnopath = "C:\\Users\\aaron\\OneDrive\\Dokumente\\Uni\\Studienarbeit\\manuelle Annotationen\\"

includepath = "C:\\Users\\aaron\\OneDrive\\Dokumente\\Uni\\Studienarbeit\\includedFiles.csv"
path = "\\\\vs-grp06.zih.tu-dresden.de\\gl4psgdata\\tsm-retro-lm\\"

saveMetricdir = 'C:\\Users\\aaron\\OneDrive\\Dokumente\\Uni\\Studienarbeit\\Metrics\\'

In [ ]:
MAnnofilenames = next(walk(MAnnopath), (None, None, []))[2] 
AAnnofilenames = next(walk(AAnnopath), (None, None, []))[2] 
startAtFileNr = AAnnofilenames.index(startfilename)+ fileoffset
stopAtFileNr = startAtFileNr + numberOfFiles
metricfilenames = next(walk(saveMetricdir), (None, None, []))[2] 

In [ ]:
#finds all LM that fullfill the AASM time requirements [5-90]s
def findRecurringLM(annotation):
    recurringLM = []
    buffer = []
    for m in range(len(annotation)-1): #collecting LM in buffer
        if (annotation[m+1][0] - annotation[m][0] > 5*sampleFreq) and (annotation[m+1][0] - annotation[m][0] < 90*sampleFreq):
            buffer.append(m)
            if(m == len(annotation)-2): #last LM ist part of series
                buffer.append(m+1)

        elif len(buffer) !=0: #end of series 
            buffer.append(m)
            recurringLM.append(buffer)
            buffer = []
            
    if len(buffer) != 0: recurringLM.append(buffer)
    return recurringLM
    
#flattens any list to 1D
def flattenx(nested_list, regular_list):
    
    for ele in nested_list:
        if type(ele) is list:
            flattenx(ele, regular_list)
        else:
            regular_list.append(ele)
    return regular_list

def flatten(nested_list):
    return flattenx(nested_list, regular_list = [])

#finds the series that a certain LM is in 
def correspondingLMSeries(LMidx, LMSeries):
    for Serie in LMSeries:
        if(LMidx in Serie):
            return Serie 
    return []

#index of the annotation signal where given LM would fit 
def FNindex(LM, Annoc):
    for idx in range(len(Annoc)):
        if(LM [0] < Annoc[idx][1]):
            return idx
    else: return len(Annoc)
    
#calculates TP,FP,TN,FN transforming the annotation signals in one second segments
def segmentwiseClasses(mAnno,aAnno):
    mannoBuffer = []
    for i in range(len(mAnno)):
        mannoBuffer = np.append(mannoBuffer, np.zeros(mAnno[i][0] - len(mannoBuffer), dtype=int))
        mannoBuffer = np.append(mannoBuffer, np.ones(mAnno[i][1] - len(mannoBuffer), dtype=int))
    mannoBuffer = np.append(mannoBuffer, np.zeros(int(len(Sleepstage)*sampleFreq - len(mannoBuffer))))
    
    msecondBins = []
    for s in range(len(Sleepstage)):
        if(len(np.where(mannoBuffer[s*sampleFreq:(s+1)*sampleFreq] == 1)[0]) > 100):#more than 50%
            msecondBins.append(1)
            
        else:
            msecondBins.append(0)
    aannoBuffer = []
    for i in range(len(aAnno)):
        aannoBuffer = np.append(aannoBuffer, np.zeros(aAnno[i][0] - len(aannoBuffer), dtype=int))
        aannoBuffer = np.append(aannoBuffer, np.ones(aAnno[i][1] - len(aannoBuffer), dtype=int))
    aannoBuffer = np.append(aannoBuffer, np.zeros(int(len(Sleepstage)*sampleFreq - len(aannoBuffer))))
    
    asecondBins = []
    for s in range(len(Sleepstage)):
        if(len(np.where(aannoBuffer[s*sampleFreq:(s+1)*sampleFreq] == 1)[0]) > 100):#more than 50%
            asecondBins.append(1)
            
        else:
            asecondBins.append(0)
    TP,FP,TN,FN = 0,0,0,0
    for s in range(len(msecondBins)):
        if(msecondBins[s] == 1 and asecondBins[s] == 1):
            TP += 1
        elif(msecondBins[s] == 0 and asecondBins[s] == 1):
            FP += 1
        elif(msecondBins[s] == 1 and asecondBins[s] == 0):
            FN += 1
        elif(msecondBins[s] == 0 and asecondBins[s] == 0):
            TN += 1
    return TP,FP,TN,FN

def calcClassicalMetrics(TP,FP,TN,FN):
    F1 = 2*TP/(2*TP+FP+FN) if TP+FP+FN != 0 else math.inf
    p0 = (TP+TN)/(TP+TN+FP+FN) if TP+TN+FP+FN != 0 else math.inf
    pe = ((TP+FN)*(TP+FP)+(TN+FP)*(TN+FN))/(TP+TN+FP+FN)**2 if TP+TN+FP+FN != 0 else math.inf
    k = (p0-pe)/(1-pe) if 1-pe != 0 else math.inf
    Spez = TN/(TN+FP) if TN+FP != 0 else math.inf
    Prec = TP/(TP+FP) if TP+FP != 0 else math.inf
    Sens = TP/(TP+FN) if TP+FN != 0 else math.inf
    Acc = (TP+TN)/(TP+TN+FP+FN) if TP+TN+FP+FN != 0 else math.inf
    NPV = TN/(TN+FN) if TN+FN != 0 else math.inf

    return F1,k,Spez,Prec,Sens,Acc,NPV



#plots two annotaitonsignals with shape [[starLM1,endLM1],[starLM2,endLM2],...]
def plotAnnotation(signals, labels,  lightOff, lightOn, plotwindow):
    fig = plt.figure(figsize=(16,2))
    plt.rcParams["figure.autolayout"] = True
    plt.margins(x=0.1, y=0.1)
    for sidx, signal in enumerate(signals):

        annoBuffer = np.zeros(lightOff, dtype=int)
        
        for i in range(len(signal)):
            annoBuffer = np.append(annoBuffer, np.zeros(signal[i][0] - len(annoBuffer), dtype=int))
            annoBuffer = np.append(annoBuffer, np.ones(signal[i][1] - len(annoBuffer), dtype=int))
        annoBuffer = np.append(annoBuffer, np.zeros(lightOn - len(annoBuffer)))
        annoBuffer = ["kein LM" if annoSample == 0 else "LM" for annoSample in annoBuffer]
        
        #offset of signals for better view
        offset = matplotlib.transforms.Affine2D().translate(0, 3*sidx)
        
        shadow_transform = fig.axes[0].transData + offset
        plt.plot(annoBuffer[plotwindow[0]:plotwindow[1]], label = labels[sidx], transform = shadow_transform)
        
    plt.xticks(np.arange(0,(plotwindow[1]-plotwindow[0])*11/10, (plotwindow[1]-plotwindow[0])/10))
    plt.xlabel("Zeit in ms")
    plt.legend(loc = "upper right")

In [ ]:
customname = "Xto1"


with open(includepath , "r", newline='') as csvfile:

    content = csv.reader(csvfile,  delimiter=';', quotechar='"')
    strings = list(content)
    data = []
    for e in strings:
        data.append(e)

includefiles = flatten(data)


totalstarttime = time.time()
for filename in ["acq_058268826.edf"]:#example to test against
#for filename in [file for file in includefiles[startAtFileNr:stopAtFileNr] if file+ ".csv" not in metricfilenames]:
    filestarttime = time.time()
    print("working on file " + filename)
    print("Nr: "+ str(includefiles.index(filename)) + " noch " +str(stopAtFileNr - includefiles.index(filename)))
    f = pyedflib.EdfReader(path + filename)
    Sleepstage = f.readSignal(2)
    sampleFreq = int(f.getSampleFrequency(0))
    f.close()

    #reading csv back to list
    with open(MAnnopath + filename + ".csv" , "r") as csvfile:
        content = csv.reader(csvfile, delimiter=',', quotechar='"')
        strings = list(content)
        if(len(strings) == 0 or strings[0][0] == " "):
            continue
        mAnno = []
        for e in strings[0]:
            mAnno.append(eval(e))


    mAnno = [LMS for LMS in mAnno if Sleepstage[int(LMS[0]/200)] != 0]

    #custom signals
    mAnno = [[10000,15000],[15500,15800],[17000,17200],[18100,19000]]
    aAnno = [[9000,18500],[19200,20300],[21400,22500],[23600,24000]] 
    #aAnno = [[10000,15000],[15500,15800],[17000,17200],[17800,19000]]#,[20000,21000]]

    '''
    calculating metrics as usual -refer to MetricfromCSV
    '''
    matchings = [] # contains indices (one for each signal) of LM that overlap
    for m in range(len(mAnno)): 
        for a in range(len(aAnno)):
            if (mAnno[m][0] < aAnno[a][1]) and ((mAnno[m][1] > aAnno[a][0])): #any overlap
                totalOverlap = (min(mAnno[m][1],aAnno[a][1]) - max(mAnno[m][0],aAnno[a][0]))
                smallerLMArea = (min(mAnno[m][1]-mAnno[m][0],aAnno[a][1]-aAnno[a][0]))
                matchings.append([m,a])
    uniqueMAnno = set()

    OneToXList = [idx for idx in [match[0] for match in matchings] if idx in uniqueMAnno or uniqueMAnno.add(idx)] 
    OneToX = len(OneToXList)+len(set(OneToXList))

    uniqueAAnno = set()
    XToOneList = [idx for idx in [match[1] for match in matchings] if idx in uniqueAAnno or uniqueAAnno.add(idx)] 
    XToOne = len(XToOneList)+len(set(XToOneList))
    
    
    # indices of mAnno - LM that are manually annotated but not automatically annotated
    FNList = []
    for idx in range(len(mAnno)):
        if idx not in [i[0] for i in matchings]:
            FNList.append(idx)
    # indices of aAnno - LM that are automatically annotated but not manually annotated
    FPList = []
    for idx in range(len(aAnno)):
        if idx not in [i[1] for i in matchings]:
            FPList.append(idx)
            
    FN = len(FNList)
    FP = len(FPList)
    TP = len(matchings) - len(XToOneList) - len(OneToXList)
    
    
    '''
    TN are counted by a rising edge in either signal that is not within a LM of the other signal
    when both signals rise together the time prior of on of the Signals counts as TN 
    (assuming there are no single Samples counting as TN)
    when one signal is rising as the other one is falling there is no TN '''
    within = 0
    for mLM in mAnno:
         within = within + [mLM[0] > aLM[0] and mLM[0] <= aLM[1] for aLM in aAnno].count(True) 

    for aLM in aAnno:
         within = within + [aLM[0] >= mLM[0] and aLM[0] <= mLM[1] for mLM in mAnno].count(True) 

    TN = len(mAnno)+len(aAnno) - within + 1
    
    SF1,Sk,SSpez,SPrec,SSens,SAcc,SNPV = 0,0,0,0,0,0,0 #calcClassicalMetrics(STP,SFP,STN,SFN)
    
    F1,k,Spez,Prec,Sens,Acc,NPV = calcClassicalMetrics(TP,FP,TN,FN)
    
    #removes the XtoOne and OneToX multiple matchings from list so that only the first and the last matched of each LM are in new List
    currentMatch = [-1,-1]

    cleanedMatchingsForFirsts = []
    for match in matchings:
        if currentMatch[0] != match[0] and currentMatch[1] != match[1]:
            cleanedMatchingsForFirsts.append(match)
        currentMatch = match


    cleanedMatchingsForLasts = []
    for i in range(len(matchings)):
        if i+1 < len(matchings):
            nextMatch = matchings[i+1] 
        else: cleanedMatchingsForLasts.append(matchings[i])
        if nextMatch[0] != matchings[i][0] and nextMatch[1] != matchings[i][1]:
            cleanedMatchingsForLasts.append(matchings[i])


    LMStartDistribution = []
    for match in cleanedMatchingsForFirsts:
        LMStartDistribution.append(mAnno[match[0]][0]-aAnno[match[1]][0])
    #plt.hist(LMStartDistribution, label='start time distribution')

    LMEndDistribution = []
    for match in cleanedMatchingsForLasts:
        LMEndDistribution.append(mAnno[match[0]][1]-aAnno[match[1]][1])

    LMStartDistribution_mean = np.mean(LMStartDistribution)
    LMEndDistribution_mean = np.mean(LMEndDistribution)

    LMStartDistribution_std = np.std(LMStartDistribution)
    LMEndDistribution_std = np.std(LMEndDistribution)
    
    mLMSeries = findRecurringLM(mAnno)
    aLMSeries = findRecurringLM(aAnno)
    
    aLMcount = len(aAnno)
    mLMcount = len(mAnno)

    mPLMcount = sum(len(series) for series in mLMSeries)
    aPLMcount = sum(len(series) for series in aLMSeries)
    
    
    realALMSeries = [series for series in aLMSeries if len(series)>=4]
    realMLMSeries = [series for series in mLMSeries if len(series)>=4]
    
    realaPLMcount = len(flatten(realALMSeries))
    realmPLMcount = len(flatten(realMLMSeries))
    
    TST = len([stage for stage in Sleepstage if stage != 0]) # in seconds

    mPLMph = realmPLMcount/TST if TST != 0 else math.inf
    aPLMph = realaPLMcount/TST if TST != 0 else math.inf
    
    mFourRecurringViolations = len(mLMSeries) - len(realMLMSeries)
    aFourRecurringViolations = len(aLMSeries) - len(realALMSeries)
    
    #mistakes in Costfunctional that have been accounted for 
    overcountedA = []
    undercountedM = []
    undercountedA = []

    # Number of PLM that are were falsely counted because of FP
    overcountedPLM = 0

    for realALMSerie in realALMSeries:
        buffer = []
        FPcounter = 0
        for FP in FPList:
            if FP in realALMSerie: 
                FPcounter += 1
                buffer.append(FP)

        if(len(realALMSerie)-FPcounter>=4):

            overcountedPLM += FPcounter
            if(FPcounter != 0):
                overcountedA.append(buffer)

        else:
            overcountedPLM += len(realALMSerie)
            overcountedA.append(realALMSerie)
    overcountedA = flatten(overcountedA)
    
    
    # first entry of OneToXMmatches is the mAnno index that the OnetoX was matched to (rest of entries)
    OneToXMmatches = []

    for One in set(OneToXList):
        buffer = [One]
        for match in matchings:
            if(match[0]== One):
                buffer.append(match[1])
        OneToXMmatches.append(buffer)

    #OneToX Overcount where a OneToX match is responsible for changing the PLM number without changing the 4 rule 
    OneToXOvercount = 0

    for rSeries in realALMSeries:
        buffer = []
        LMcounter = 0 
        for rLM in rSeries:#count the onetox responsible for each series 
            if(rLM in flatten([matches[1:] for matches in OneToXMmatches])):
                LMcounter += 1
                buffer.append(rLM)

        if(len(rSeries)-LMcounter >= 4): # case onetox did not change the 4 rule 
            OneToXOvercount += LMcounter
            if(LMcounter != 0):
                overcountedA.append(buffer)
        else: #change due to 4 rule 
            OneToXOvercount += len(rSeries)
            overcountedA.append(rSeries)
    overcountedA = flatten(overcountedA)
    
    #TP that are in realALMSerie while corresponding match is not in realMLMSeries
    TPTimeViolationsOvercount = 0 

    #regular 
    for realALMSerie in realALMSeries:
        buffer = []

        for LM in realALMSerie:
            for match in matchings:#find match
                if(match[0] not in flatten(OneToXList) and match[1] == LM and match[0] not in flatten(correspondingLMSeries(LM, mLMSeries))):
                    buffer.append(match[1])
        if(len(realALMSerie)-len(buffer) < 4):
            overcountedA.append(realALMSerie)
            TPTimeViolationsOvercount += len(realALMSerie)

        else:
            overcountedA.append(buffer)
            TPTimeViolationsOvercount += len(buffer)
    overcountedA = flatten(overcountedA)
   
    
    #Xto1 matching that falsely reduces PLM count
    XToOneAmatches = []

    for One in set(XToOneList):
        buffer = [One]
        for match in matchings:
            if(match[1] == One):
                buffer.append(match[0])
        XToOneAmatches.append(buffer)

    XToOneUndercount = 0

    for rSeries in realMLMSeries:
        LMcounter = 0 
        for rLM in rSeries:#count the xto1 responsible for each series 
            if(rLM in flatten([matches[1:] for matches in XToOneAmatches])):
                LMcounter += 1
        if(len(rSeries)-LMcounter >= 4): # case onetox did not change the 4 rule 
            XToOneUndercount += LMcounter
            if(LMcounter != 0):
                undercountedM.append(rLM)
        else: #change due to 4 rule 
            XToOneUndercount += len(rSeries)
            undercountedM.append(rSeries)
    undercountedM = flatten(undercountedM)
    
    
    
    TPTimeViolationsUndercount = 0 
    #regular 
    for realMLMSerie in realMLMSeries:
        buffer = []

        for LM in realMLMSerie:
            for match in matchings:#find match
                if(match[1] not in flatten(XToOneList) and match[0] == LM and match[1] not in flatten(correspondingLMSeries(LM, aLMSeries))):
                    buffer.append(match[1])
        if(len(realMLMSerie)-len(buffer) < 4):
            undercountedM.append(realMLMSerie)
            TPTimeViolationsUndercount += len(realMLMSerie)

        else:
            undercountedA.append(buffer)
            TPTimeViolationsUndercount += len(buffer)

    undercountedM = flatten(undercountedM)
    undercountedA = flatten(undercountedA)
    
    # Number of PLM that are were not counted beacuse of FN
    undercountedPLM = 0

    aAnnoc = copy.deepcopy(aAnno)

    oldRecurring = [Serie for Serie in findRecurringLM(aAnnoc) if len(Serie) >= 4]
    oldLen = len(flatten(oldRecurring))

    for FN in FNList:
        aAnnoc.insert(FNindex(mAnno[FN],aAnnoc),mAnno[FN])
    newRecurring = [Serie for Serie in findRecurringLM(aAnnoc) if len(Serie) >= 4]
    undercountedPLM =  len(flatten(newRecurring)) - oldLen

    for Serie in newRecurring:
        FNcounter = 0
        FNbuffer = []
        for indx,newLM in enumerate(aAnnoc):
            if( newLM in mAnno and mAnno.index(newLM) in FNList):
                FNcounter += 1
                FNbuffer.append(mAnno.index(newLM))
                
        if(len(Serie)-FNcounter<4): #Series founded
            for LM in Serie:
                undercountedM.append(mAnno.index(aAnnoc[LM]))
        else:
            undercountedM.append(FNbuffer)
                        
    undercountedM = flatten(undercountedM)
    
    doubleOverCount = len(overcountedA)-len(set(overcountedA))
    doubleUnderCount = len(undercountedM)-len(set(undercountedM))+len(undercountedA)-len(set(undercountedA))
    
    
    
    #cost functional 
    K = overcountedPLM + OneToXOvercount + TPTimeViolationsOvercount - doubleOverCount + (XToOneUndercount + TPTimeViolationsUndercount + undercountedPLM - doubleUnderCount)  
    Krel = K/realmPLMcount if realmPLMcount != 0 else math.inf
    metrics = [TP,TN,FP,FN,F1,k,Spez,Prec,Sens,Acc,NPV,SF1,Sk,SSpez,SPrec,SSens,SAcc,SNPV,mLMcount,aLMcount,mPLMcount,aPLMcount,realmPLMcount,realaPLMcount,OneToX,XToOne,LMStartDistribution_mean,LMEndDistribution_mean,LMStartDistribution_std,LMEndDistribution_std,mFourRecurringViolations,aFourRecurringViolations, doubleOverCount,doubleUnderCount,overcountedPLM,OneToXOvercount,TPTimeViolationsOvercount,XToOneUndercount,TPTimeViolationsUndercount,undercountedPLM,K, Krel,mPLMph,aPLMph]
    metrics.append("custom"+ customname)
     
    if(savemetrics):
        with open("custom"+ customname +  ".csv", "w") as csvfile:
            swriter = csv.writer(csvfile, delimiter=',')
            swriter.writerow(metrics)
    fileendtime = time.time()
    print("runningtime: "+'{:5.3f}s'.format(fileendtime-filestarttime))

In [ ]:
plotAnnotation([mAnno,aAnno],["manuelle Annotation","automatische Annotation"] ,  8000,28000, [8000,28000])

displayMetric(metrics)